In [1]:
# Import libraries
import numpy as np
import pandas as pd
import tldextract
import os
import pickle
from bs4 import Comment
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import time



In [2]:
class html_check():
    def __init__(self,text,url):  ##input the html and the domain name for example
                                     
        self.text = text
        self.url = url
        self.domain = tldextract.extract(self.url).domain
        
#------------------------------length of HTML----------------------------------------------      
    
    def find_len(self,tag):
        if tag =='!--':
            soup = self.text
            len_sum = 0
            for comment in soup.find_all(string=lambda text:isinstance(text, Comment)):
                len_sum += len(comment)
            return len_sum          
        else:
            soup = self.text
            scripts = soup.find_all(str(tag))
            len_sum = 0

            for script in scripts:
                  len_sum += len(script.text)
            return len_sum
        
    def len_html_tag(self):  ## this is the total length for 5 special tags
        return html_check.find_len(self,"style") + html_check.find_len(self,"link") + html_check.find_len(self,"form") + html_check.find_len(self,"!--") + html_check.find_len(self,"script")

    def len_html(self):
        return len(self.text.text)
    
#------------------------------hidden content------------------------------------------------------------------------
    def hidden_div(self):
        soup = self.text
        scripts = soup.find_all('div')          
        find = 0
        for script in scripts:
            try:
                if str(script.attrs['style'])=='visibility:hidden' or str(script.attrs['style'])=='display:none':
                    find = 1    
                    break               
            except:
                continue
        return find 
    
    def hidden_button(self):
        soup = self.text
        scripts = soup.find_all('button')          
        find = 0
        for script in scripts:
            try:
                if str(script.attrs['disabled'])=='disabled':
                    find = 1   
                    break
            except:
                continue
        return find 
    
    def hidden_input(self):
        soup = self.text
        scripts = soup.find_all('input')          
        find = 0
        for script in scripts:
            try:
                if str(script.attrs['type'])=='hidden' or str(script.attrs['disabled'])=='disabled':
                    find = 1
                    break
            except:
                continue                
        return find 
    
  
    def hidden(self): ## have hidden content
        return int(html_check.hidden_div(self) | html_check.hidden_button(self) | html_check.hidden_input(self))
    
#----------------------------link based------------------------------------------------    

    def find_all_link(self):
        soup = self.text
        a_tags = soup.find_all('a')
        a_data = []

        for a_tag in a_tags:
            try:
                a_data.append(a_tag.attrs['href'])
            except:
                continue
        
        return a_data
    
    
    def find_source(self,tag):  ## find src attribute in <img> <link> ... 
        if tag =='link':
            soup = self.text
            links = soup.find_all('link')
            link_data = []

            for link in links:
                try:
                    link_data.append(link.attrs['href'])
                except:
                    continue
            return link_data
        
        else:
            soup = self.text
            resources = soup.find_all(str(tag))
            data = []

            for resource in resources:
                try:
                    data.append(resource.attrs['src'])       
                except:
                    continue
            return data
        
    def internal_external_link(self):  ## Number of internal hyperlinks and number of external hyperlinks
        link_list = html_check.find_all_link(self)
        if len(link_list)==0:  ## in case there is no hyperlink
            return [0, 0]  
        
        count = 0
        for j in link_list:
            if "http" in j:
                brand = tldextract.extract(j).domain
                if str(brand) == self.domain:
                    count += 1
            else:
                count +=1                
       
        return [count, len(link_list) - count]

    def empty_link(self): ## Number of empty links
        link_list = html_check.find_all_link(self)
        count = 0
        for j in link_list:
            if j=="" or j=="#" or j=='#javascript::void(0)' or j=='#content' or j=='#skip' or j=='javascript:;' or j=='javascript::void(0);' or j=='javascript::void(0)':
                count += 1
        if len(link_list)==0:
            return 0
        return count 

#----------------------------form based---------------------------------------------------    
    
    def find_form(self):
        soup = self.text
        forms = soup.find_all('form')
        data = []

        for form in forms:
            input_tags = form.find_all('input')
            for input_ in input_tags:
                try:
                    if input_.has_key('name'):
                        data.append(str(input_['name']))
                except:
                    continue
        
        return data
    
    def login_form(self): ## have login-form requires password
        input_list = html_check.find_form(self)
        result = 0
        for j in input_list:
            if j.find("password")!=-1 or j.find("pass")!=-1 or j.find("login")!=-1 or j.find("signin")!=-1:
                result = 1
                break
        return result
    
    
    def internal_external_resource(self): ##
        tag_list = ['link','img','script','noscript']
        resource_list = []
        count = 0
        for tag in tag_list:
            resource_list.append(html_check.find_source(self,tag))
        
        resource_list = [y for x in resource_list for y in x]
        if len(resource_list)==0: ## in case there is no resource link
            return [0, 0]

        for j in resource_list:
            if "http" in j:
                if not(self.domain == tldextract.extract(j).domain):
                    count +=1   
        
        return len(resource_list) - count, count
    
    
#-----------------suspicious element HTML-------------------------------------------------        
     
    def redirect(self): ##auto-refresh webpage
        soup = self.text
        return int('redirect' in soup)   
                                  
    def alarm_window(self):  ## alert window pop up
        soup = self.text
        scripts = soup.find_all('script')          
        find = 0
        for script in scripts:
            try:
                if('alert' in str(script.contents)) or ('window.open' in str(script.contents)):
                    find = 1
                    break  
            except:
                continue

        return find
#---------------------------------domain vs HTML content----------------------------------------
    def title_domain(self):
        soup = self.text
        try:
            return int(self.domain.lower() in soup.title.text.lower())
        except:
            return 0
        
    def domain_occurrence(self):
        try:
            return str(self.text).count(self.domain)
        except:
            return 0
        
    def brand_freq_domain(self):
        link_list = html_check.find_all_link(self)
        domain_list = []
        
        for j in link_list:
            if "http" in j:
                brand = tldextract.extract(j).domain
                domain_list.append(brand)
            else:
                domain_list.append(self.domain)
              
        if len(domain_list) == 0:
            return 1
        if pd.Series(domain_list).value_counts().index[0] == self.domain:
            return 1
        else:
            return 0


In [3]:

class URL_check():
    
    def __init__(self, url, tldlist_path):
        self.url = url.lower()
        self.tldlist_path = tldlist_path
    def domain_is_IP(self):
        
        if len(tldextract.extract(self.url).subdomain) == 0:
            hostname = tldextract.extract(self.url).domain
        else:
            hostname = '.'.join([tldextract.extract(self.url).subdomain, tldextract.extract(self.url).domain])
          
        if np.sum([i.isdigit() for i in hostname.split(".")])==4:
            return 1
        else:
            return 0   

    def symbol_count(self):
        
        punctuation_list = ["@", '-' ,'~']
        count=0
        for j in punctuation_list:
            if j in self.url:
                count+=1
        return count

    def https(self):
        return int("https://" in self.url)
    
    def domain_len(self):
        if len(tldextract.extract(self.url).subdomain)==0:
            domain_len = len(tldextract.extract(self.url).domain+"."+tldextract.extract(self.url).suffix)
        else:
            domain_len=len(tldextract.extract(self.url).subdomain +"."+ tldextract.extract(self.url).domain+"."+tldextract.extract(self.url).suffix)
        return domain_len
    
    def url_len(self):
        return len(self.url)

    def num_dot_hostname(self):
        if len(tldextract.extract(self.url).subdomain) == 0:
            hostname = tldextract.extract(self.url).domain
        else:
            hostname = '.'.join([tldextract.extract(self.url).subdomain, tldextract.extract(self.url).domain])
        return hostname.count('.')
    
    def sensitive_word(self):
        sensitive_list = ["secure", "account", "webscr", "login",
                          "signin", "ebayisapi", "banking", "confirm"]
        return int(any(x in self.url for x in sensitive_list))
 
    def tld_in_domain(self):
        path = self.tldlist_path
        tld_list = list(pd.read_csv(path, encoding = "ISO-8859-1").Domain.apply(lambda x: x.replace('.','')))
        
        for tld in tld_list: 
            if tld == tldextract.extract(self.url).subdomain or tld == tldextract.extract(self.url).domain:
                return 1
        return 0
        
    def tld_in_path(self):
        path = self.tldlist_path
        tld_list = list(pd.read_csv(path, encoding = "ISO-8859-1").Domain.apply(lambda x: x.replace('.','')))
        
        for tld in tld_list:
            if tld == urlparse(self.url).path or tld == urlparse(self.url).params or tld == urlparse(self.url).query or tld == urlparse(self.url).fragment:
                return 1
        return 0

In [ ]:
import h5py
from tqdm.notebook import tqdm
import pandas as pd
import os
import time
import pickle
from bs4 import BeautifulSoup
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Set default parameters
defaults = {
    'output_basedir': 'TestResults',
    'modeldir': 'model_directory',
    'tldlist': 'tld.csv'
}

# Set batch size
batch_size = 500

print('loading h5py')

# Extract HTML features
internal_link = []
external_link = []
empty_link = []
login_form = []
html_len_tag = []
html_len = []
alarm_window = []
redirection = []
hidden = []
title_domain = []
brand_domain = []
internal_resource = []
external_resource = []
domain_occurrence = []

# Extract URL features
domain_is_ip = []
symbol_count = []
http = []
domain_len = []
url_len = []
num_dot_hostname = []
sensitive_word = []
tld_in_domain = []
tld_in_path = []

phish = []

# Load data in batches
with h5py.File('../data/phishing_output.h5', 'r') as h5_file:
    # Pull 'urls' and 'labels' from the 'test' dataset
    total_entries = len(h5_file['train/urls'])
    
    count = 0
    for start in range(0, total_entries, batch_size):
        count += 1
        print(count)
        end = min(start + batch_size, total_entries)
        urls_batch = [url.decode('utf-8') for url in h5_file['train/urls'][start:end]]
        raw_html_content_batch = h5_file['train/html_content'][start:end]
        labels_batch = h5_file['train/labels'][start:end]

        # Process HTML content in the current batch
        content = []
        for i in range(len(raw_html_content_batch)):
            try:
                with open(raw_html_content_batch[i]) as html_content:
                    soup = BeautifulSoup(html_content, "html5lib")
                    if len(soup) == 0:
                        content.append('')
                    else:
                        content.append(soup)
            except Exception as e:
                content.append('')


        for i in range(len(content)):
            phish.append(labels_batch[i])
            if len(content[i]) == 0:
                internal_link.append(0)
                external_link.append(0)
                empty_link.append(0)
                login_form.append(0)

                html_len_tag.append(0)
                html_len.append(0)

                alarm_window.append(0)
                redirection.append(0)
                hidden.append(0)

                title_domain.append(0)
                internal_resource.append(0)
                external_resource.append(0)
                domain_occurrence.append(0)
                brand_domain.append(0)
            else:
                test = html_check(content[i], urls_batch[i])
                internal_link.append(test.internal_external_link()[0])
                external_link.append(test.internal_external_link()[1])
                empty_link.append(test.empty_link())
                login_form.append(test.login_form())

                html_len_tag.append(test.len_html_tag())
                html_len.append(test.len_html())

                alarm_window.append(test.alarm_window())
                redirection.append(test.redirect())
                hidden.append(test.hidden())

                title_domain.append(test.title_domain())
                internal_resource.append(test.internal_external_resource()[0])
                external_resource.append(test.internal_external_resource()[1])
                domain_occurrence.append(test.domain_occurrence())
                brand_domain.append(test.brand_freq_domain())

        for i in range(len(urls_batch)):
            url_class = URL_check(urls_batch[i], defaults['tldlist'])
            domain_is_ip.append(url_class.domain_is_IP())
            symbol_count.append(url_class.symbol_count())
            http.append(url_class.https())
            domain_len.append(url_class.domain_len())
            url_len.append(url_class.url_len())
            num_dot_hostname.append(url_class.num_dot_hostname())
            sensitive_word.append(url_class.sensitive_word())
            tld_in_domain.append(url_class.tld_in_domain())
            tld_in_path.append(url_class.tld_in_path())






loading h5py
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
108
109
110
111
112
113
114


In [7]:
# Combine features into a dataframe
import xgboost as xgb
import _pickle as pickle
import pandas as pd
import joblib
from sklearn.model_selection import KFold,train_test_split,GridSearchCV
from sklearn.metrics import confusion_matrix, mean_squared_error,accuracy_score
from sklearn.datasets import load_iris,load_digits,load_boston
import numpy
import matplotlib.pyplot as plt

df_feature = pd.concat([
    pd.Series(internal_link), pd.Series(external_link), pd.Series(empty_link),
    pd.Series(login_form), pd.Series(html_len_tag), pd.Series(html_len),
    pd.Series(alarm_window), pd.Series(redirection), pd.Series(hidden),
    pd.Series(title_domain), pd.Series(brand_domain), pd.Series(internal_resource),
    pd.Series(external_resource), pd.Series(domain_occurrence), pd.Series(domain_is_ip),
    pd.Series(symbol_count), pd.Series(http), pd.Series(domain_len),
    pd.Series(url_len), pd.Series(num_dot_hostname), pd.Series(sensitive_word),
    pd.Series(tld_in_domain), pd.Series(tld_in_path), pd.Series(phish)
], axis=1)

df_feature.columns = ['internal_link', 'external_link', 'empty_link',
                      'login_form', 'html_len_tag', 'html_len',
                      'alarm_window', 'redirection', 'hidden',
                      'title_domain', 'brand_domain', 'internal_resource', 'external_resource',
                      'domain_occurrence', 'domain_is_ip', 'symbol_count', 'http',
                      'domain_len', 'url_len', 'num_dot_hostname', 'sensitive_word',
                      'tld_in_domain', 'tld_in_path', 'phish']

y_train = df_feature['phish']
x_train = df_feature.drop(columns=['phish'])


# Training XGBoost model
print(len(x_train))
print(list(y_train).count(1))

xgb_model = xgb.XGBClassifier()
xgb_model.fit(x_train, y_train)
joblib.dump(xgb_model, 'my-model.pkl')




56635
21296


['my-model.pkl']

In [ ]:
import h5py
from tqdm.notebook import tqdm
import pandas as pd
import os
import time
import pickle
from bs4 import BeautifulSoup
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Set default parameters
defaults = {
    'output_basedir': 'TestResults',
    'modeldir': 'model_directory',
    'tldlist': 'tld.csv'
}

# Set batch size
batch_size = 100

# Initialize counters for metrics
true_positives = 0
true_negatives = 0
false_positives = 0
false_negatives = 0

def update_metrics(y_true, y_pred):
    global true_positives, true_negatives, false_positives, false_negatives
    for true, pred in zip(y_true, y_pred):
        if true == 1 and pred == 1:
            true_positives += 1
        elif true == 0 and pred == 0:
            true_negatives += 1
        elif true == 0 and pred == 1:
            false_positives += 1
        elif true == 1 and pred == 0:
            false_negatives += 1

            
print('loading h5py')

# Load data in batches
with h5py.File('../data/phishing_output.h5', 'r') as h5_file:
    # Pull 'urls' and 'labels' from the 'test' dataset
    total_entries = len(h5_file['producthunt/urls'])
    
    for start in range(0, total_entries, batch_size):
        end = min(start + batch_size, total_entries)
        urls_batch = [url.decode('utf-8') for url in h5_file['producthunt/urls'][start:end]]
        raw_html_content_batch = h5_file['producthunt/html_content'][start:end]
        labels_batch = h5_file['producthunt/labels'][start:end]

        # Process HTML content in the current batch
        content = []
        for i in range(len(raw_html_content_batch)):
            try:
                with open(raw_html_content_batch[i]) as html_content:
                    soup = BeautifulSoup(html_content, "html5lib")
                    if len(soup) == 0:
                        content.append('')
                    else:
                        content.append(soup)
            except Exception as e:
                content.append('')

        # Extract HTML features
        internal_link = []
        external_link = []
        empty_link = []
        login_form = []
        html_len_tag = []
        html_len = []
        alarm_window = []
        redirection = []
        hidden = []
        title_domain = []
        brand_domain = []
        internal_resource = []
        external_resource = []
        domain_occurrence = []

        for i in range(len(content)):
            if len(content[i]) == 0:
                internal_link.append(0)
                external_link.append(0)
                empty_link.append(0)
                login_form.append(0)

                html_len_tag.append(0)
                html_len.append(0)

                alarm_window.append(0)
                redirection.append(0)
                hidden.append(0)

                title_domain.append(0)
                internal_resource.append(0)
                external_resource.append(0)
                domain_occurrence.append(0)
                brand_domain.append(0)
            else:
                test = html_check(content[i], urls_batch[i])
                internal_link.append(test.internal_external_link()[0])
                external_link.append(test.internal_external_link()[1])
                empty_link.append(test.empty_link())
                login_form.append(test.login_form())

                html_len_tag.append(test.len_html_tag())
                html_len.append(test.len_html())

                alarm_window.append(test.alarm_window())
                redirection.append(test.redirect())
                hidden.append(test.hidden())

                title_domain.append(test.title_domain())
                internal_resource.append(test.internal_external_resource()[0])
                external_resource.append(test.internal_external_resource()[1])
                domain_occurrence.append(test.domain_occurrence())
                brand_domain.append(test.brand_freq_domain())

        # Extract URL features
        domain_is_ip = []
        symbol_count = []
        http = []
        domain_len = []
        url_len = []
        num_dot_hostname = []
        sensitive_word = []
        tld_in_domain = []
        tld_in_path = []

        for i in range(len(urls_batch)):
            url_class = URL_check(urls_batch[i], defaults['tldlist'])
            domain_is_ip.append(url_class.domain_is_IP())
            symbol_count.append(url_class.symbol_count())
            http.append(url_class.https())
            domain_len.append(url_class.domain_len())
            url_len.append(url_class.url_len())
            num_dot_hostname.append(url_class.num_dot_hostname())
            sensitive_word.append(url_class.sensitive_word())
            tld_in_domain.append(url_class.tld_in_domain())
            tld_in_path.append(url_class.tld_in_path())

        # Combine features into a dataframe
        df_feature = pd.concat([
            pd.Series(internal_link), pd.Series(external_link), pd.Series(empty_link),
            pd.Series(login_form), pd.Series(html_len_tag), pd.Series(html_len),
            pd.Series(alarm_window), pd.Series(redirection), pd.Series(hidden),
            pd.Series(title_domain), pd.Series(brand_domain), pd.Series(internal_resource),
            pd.Series(external_resource), pd.Series(domain_occurrence), pd.Series(domain_is_ip),
            pd.Series(symbol_count), pd.Series(http), pd.Series(domain_len),
            pd.Series(url_len), pd.Series(num_dot_hostname), pd.Series(sensitive_word),
            pd.Series(tld_in_domain), pd.Series(tld_in_path)
        ], axis=1)

        df_feature.columns = ['internal_link', 'external_link', 'empty_link',
                              'login_form', 'html_len_tag', 'html_len',
                              'alarm_window', 'redirection', 'hidden',
                              'title_domain', 'brand_domain', 'internal_resource', 'external_resource',
                              'domain_occurrence', 'domain_is_ip', 'symbol_count', 'http',
                              'domain_len', 'url_len', 'num_dot_hostname', 'sensitive_word',
                              'tld_in_domain', 'tld_in_path']

        df_feature.index = urls_batch
        df_feature.to_csv(os.path.join(defaults['output_basedir'], f'feature_batch_{start}.csv'))

        # Get predictions
        x = df_feature

        y_prob = xgb_model.predict_proba(x)[:, 1]
        y_pred = xgb_model.predict(x)

        # Update metrics
        update_metrics(labels_batch, y_pred)

        # Output metrics for the current batch
        print("Batch {}: True Positives: {}, True Negatives: {}, False Positives: {}, False Negatives: {}"
              .format(start, true_positives, true_negatives, false_positives, false_negatives))

        pred_df = pd.DataFrame({'URL': urls_batch, 'y_pred': y_pred, 'y_prob': y_prob})
        print("Number of predicted positives in batch {}: {}".format(start, list(pred_df['y_pred']).count(1)))
        pred_df.to_csv(os.path.join(defaults['output_basedir'], f'predict_batch_{start}.csv'), index=None)


# Create the predicted and actual labels based on the counts
y_true = [1] * true_positives + [0] * false_negatives + [1] * false_negatives + [0] * true_negatives
y_pred = [1] * (true_positives + false_positives) + [0] * (false_negatives + true_negatives)

# Calculate metrics
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
accuracy = accuracy_score(y_true, y_pred)

# Output the stats
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"Accuracy: {accuracy}")

loading h5py
Batch 0: True Positives: 0, True Negatives: 92, False Positives: 8, False Negatives: 0
Number of predicted positives in batch 0: 8
Batch 100: True Positives: 0, True Negatives: 175, False Positives: 25, False Negatives: 0
Number of predicted positives in batch 100: 17
Batch 200: True Positives: 0, True Negatives: 264, False Positives: 36, False Negatives: 0
Number of predicted positives in batch 200: 11
Batch 300: True Positives: 0, True Negatives: 349, False Positives: 51, False Negatives: 0
Number of predicted positives in batch 300: 15


With Pretrained



In [ ]:
import h5py
from tqdm.notebook import tqdm
import pandas as pd
import os
import time
import pickle
from bs4 import BeautifulSoup
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Set default parameters
defaults = {
    'output_basedir': 'TestResults',
    'modeldir': 'model_directory',
    'tldlist': 'tld.csv'
}

# Set batch size
batch_size = 100

# Initialize counters for metrics
true_positives = 0
true_negatives = 0
false_positives = 0
false_negatives = 0

def update_metrics(y_true, y_pred):
    global true_positives, true_negatives, false_positives, false_negatives
    for true, pred in zip(y_true, y_pred):
        if true == 1 and pred == 1:
            true_positives += 1
        elif true == 0 and pred == 0:
            true_negatives += 1
        elif true == 0 and pred == 1:
            false_positives += 1
        elif true == 1 and pred == 0:
            false_negatives += 1

with open(os.path.join('../data/stack-model.pkl'), 'rb') as handle:
    model = pickle.load(handle)
        
            
print('loading h5py')

# Load data in batches
with h5py.File('../data/phishing_output.h5', 'r') as h5_file:
    # Pull 'urls' and 'labels' from the 'test' dataset
    total_entries = len(h5_file['test/urls'])
    
    for start in range(0, total_entries, batch_size):
        end = min(start + batch_size, total_entries)
        urls_batch = [url.decode('utf-8') for url in h5_file['producthunt/urls'][start:end]]
        raw_html_content_batch = h5_file['producthunt/html_content'][start:end]
        labels_batch = h5_file['producthunt/labels'][start:end]

        # Process HTML content in the current batch
        content = []
        for i in range(len(raw_html_content_batch)):
            try:
                with open(raw_html_content_batch[i]) as html_content:
                    soup = BeautifulSoup(html_content, "html5lib")
                    if len(soup) == 0:
                        content.append('')
                    else:
                        content.append(soup)
            except Exception as e:
                content.append('')

        # Extract HTML features
        internal_link = []
        external_link = []
        empty_link = []
        login_form = []
        html_len_tag = []
        html_len = []
        alarm_window = []
        redirection = []
        hidden = []
        title_domain = []
        brand_domain = []
        internal_resource = []
        external_resource = []
        domain_occurrence = []

        for i in range(len(content)):
            if len(content[i]) == 0:
                internal_link.append(0)
                external_link.append(0)
                empty_link.append(0)
                login_form.append(0)

                html_len_tag.append(0)
                html_len.append(0)

                alarm_window.append(0)
                redirection.append(0)
                hidden.append(0)

                title_domain.append(0)
                internal_resource.append(0)
                external_resource.append(0)
                domain_occurrence.append(0)
                brand_domain.append(0)
            else:
                test = html_check(content[i], urls_batch[i])
                internal_link.append(test.internal_external_link()[0])
                external_link.append(test.internal_external_link()[1])
                empty_link.append(test.empty_link())
                login_form.append(test.login_form())

                html_len_tag.append(test.len_html_tag())
                html_len.append(test.len_html())

                alarm_window.append(test.alarm_window())
                redirection.append(test.redirect())
                hidden.append(test.hidden())

                title_domain.append(test.title_domain())
                internal_resource.append(test.internal_external_resource()[0])
                external_resource.append(test.internal_external_resource()[1])
                domain_occurrence.append(test.domain_occurrence())
                brand_domain.append(test.brand_freq_domain())

        # Extract URL features
        domain_is_ip = []
        symbol_count = []
        http = []
        domain_len = []
        url_len = []
        num_dot_hostname = []
        sensitive_word = []
        tld_in_domain = []
        tld_in_path = []

        for i in range(len(urls_batch)):
            url_class = URL_check(urls_batch[i], defaults['tldlist'])
            domain_is_ip.append(url_class.domain_is_IP())
            symbol_count.append(url_class.symbol_count())
            http.append(url_class.https())
            domain_len.append(url_class.domain_len())
            url_len.append(url_class.url_len())
            num_dot_hostname.append(url_class.num_dot_hostname())
            sensitive_word.append(url_class.sensitive_word())
            tld_in_domain.append(url_class.tld_in_domain())
            tld_in_path.append(url_class.tld_in_path())

        # Combine features into a dataframe
        df_feature = pd.concat([
            pd.Series(internal_link), pd.Series(external_link), pd.Series(empty_link),
            pd.Series(login_form), pd.Series(html_len_tag), pd.Series(html_len),
            pd.Series(alarm_window), pd.Series(redirection), pd.Series(hidden),
            pd.Series(title_domain), pd.Series(brand_domain), pd.Series(internal_resource),
            pd.Series(external_resource), pd.Series(domain_occurrence), pd.Series(domain_is_ip),
            pd.Series(symbol_count), pd.Series(http), pd.Series(domain_len),
            pd.Series(url_len), pd.Series(num_dot_hostname), pd.Series(sensitive_word),
            pd.Series(tld_in_domain), pd.Series(tld_in_path)
        ], axis=1)

        df_feature.columns = ['internal_link', 'external_link', 'empty_link',
                              'login_form', 'html_len_tag', 'html_len',
                              'alarm_window', 'redirection', 'hidden',
                              'title_domain', 'brand_domain', 'internal_resource', 'external_resource',
                              'domain_occurrence', 'domain_is_ip', 'symbol_count', 'http',
                              'domain_len', 'url_len', 'num_dot_hostname', 'sensitive_word',
                              'tld_in_domain', 'tld_in_path']

        df_feature.index = urls_batch
        df_feature.to_csv(os.path.join(defaults['output_basedir'], f'feature_batch_{start}.csv'))

        # Get predictions
        x = df_feature

        y_prob = model.predict_proba(x)[:, 1]
        y_pred = model.predict(x)

        # Update metrics
        update_metrics(labels_batch, y_pred)

        # Output metrics for the current batch
        print("Batch {}: True Positives: {}, True Negatives: {}, False Positives: {}, False Negatives: {}"
              .format(start, true_positives, true_negatives, false_positives, false_negatives))

        pred_df = pd.DataFrame({'URL': urls_batch, 'y_pred': y_pred, 'y_prob': y_prob})
        print("Number of predicted positives in batch {}: {}".format(start, list(pred_df['y_pred']).count(1)))
        pred_df.to_csv(os.path.join(defaults['output_basedir'], f'predict_batch_{start}.csv'), index=None)


# Create the predicted and actual labels based on the counts
y_true = [1] * true_positives + [0] * false_negatives + [1] * false_negatives + [0] * true_negatives
y_pred = [1] * (true_positives + false_positives) + [0] * (false_negatives + true_negatives)

# Calculate metrics
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)
accuracy = accuracy_score(y_true, y_pred)

# Output the stats
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"Accuracy: {accuracy}")